In [3]:
from scipy.spatial import Delaunay
import json
import matplotlib.pyplot as plt
import numpy as np
from skimage.draw import polygon, polygon2mask
from scipy.interpolate import griddata
import skimage.io as io
import skimage as sk

In [4]:
stadium1 = io.imread('images/stadium_resized_1.png')
stadium2 = io.imread('images/stadium_resized_2.png')

In [5]:
f = open("stadidum_resized_1_stadium_resized_2.json")
json_data = json.load(f)
im1_points = np.array(json_data['im1Points'])
im2_points = np.array(json_data['im2Points']) 

In [6]:
def computeH(im1_points, im2_points):
  A = []
  B = []
  for i in range(len(im1_points)):
    p1 = im1_points[i]
    p2 = im2_points[i] 
    A.append([p1[0], p1[1], 1, 0, 0, 0, -p1[0] * p2[0], -p1[1] * p2[0]])
    A.append([0, 0, 0, p1[0], p1[1], 1, -p1[0] * p2[1], -p1[1] * p2[1]])
    B.append(p2[0])
    B.append(p2[1])
  A = np.asarray(A)
  B = np.asarray(B)
  H_flatten = np.linalg.lstsq(A, B)[0]
  H_flatten = np.asarray(H_flatten.tolist() + [1])
  return np.reshape(H_flatten, (3, 3))

In [7]:
H = computeH(im1_points, im2_points)

/var/folders/lf/lddj8h6s3m3g8fkfp14cq4bw0000gn/T/ipykernel_71220/628019653.py:13: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  H_flatten = np.linalg.lstsq(A, B)[0]


In [8]:
def normalize(im):
    return (im - np.amin(im)) / (np.amax(im) - np.amin(im))

In [9]:
def createGrid(xmin, xmax,ymin,ymax):
    x_matrix = np.repeat(np.arange(xmin, xmax), (ymax - ymin)).reshape(-1, (ymax - ymin)).T
    y_matrix = np.repeat(np.arange(ymin, ymax), (xmax - xmin)).reshape((ymax - ymin), (xmax - xmin))
    return x_matrix, y_matrix

In [10]:
def warpImage(im,H):
    corners = [np.array([0,0, 1]),np.array([0,im.shape[0]-1, 1]),np.array([im.shape[1]-1,0, 1]),np.array([im.shape[1]-1,im.shape[0]-1, 1])]
    warped_corners = [ corner @ H.T for corner in corners] 
    min_x, min_y = np.min(warped_corners, axis=0)[:2]
    min_x_index, min_y_index = 0, 0 
    min_x_sofar, min_y_sofar = warped_corners[0][0], warped_corners[0][1]
    max_x_index, max_y_index = 0, 0
    max_x_sofar, max_y_sofar = warped_corners[0][0], warped_corners[0][1]
    for i in range(len(warped_corners)):
        if warped_corners[i][0] < min_x_sofar:
            min_x_index = i
            min_x_sofar = warped_corners[i][0]
        if warped_corners[i][0] > max_x_sofar:
            max_x_index = i
            max_x_sofar = warped_corners[i][0]
        if warped_corners[i][1] < min_y_sofar:
            min_y_index = i
            min_y_sofar = warped_corners[i][1]
        if warped_corners[i][1] > max_y_sofar:
            max_y_index = i
            max_y_sofar = warped_corners[i][1]
    min_x, min_y = np.floor(min_x / warped_corners[min_x_index][2]), np.floor(min_y / warped_corners[min_y_index][2])
    max_x, max_y = np.max(warped_corners, axis=0)[:2]
    max_x, max_y = np.ceil(max_x / warped_corners[max_x_index][2]), np.ceil(max_y / warped_corners[max_y_index][2])
    warped_corners = np.asanyarray(warped_corners)
    new_width = int(abs(min_x - max_x))
    new_height = int(abs(min_y - max_y)) 
    x_coords, y_coords = createGrid(int(min_x), int(max_x), int(min_y), int(max_y))
    coords = np.stack((x_coords, y_coords, np.ones(x_coords.shape)), axis = 2)
    coords = coords.reshape((x_coords.size, 3))
    inv_warp_x, inv_warp_y, _ = coords.T
    img_gridx,img_gridy = createGrid(0, im.shape[1], 0, im.shape[0])
    img_coords = np.stack((img_gridx, img_gridy, np.ones(img_gridx.shape)), axis = 2)
    img_coords = img_coords.reshape((img_gridx.size, 3))
    img_x, img_y, _ = img_coords.T 
    interpret_values = [] 
    for i in range(im.shape[2]):
        temp = griddata((img_x, img_y), im[:, :, i].flatten(), (inv_warp_x, inv_warp_y), method='linear', fill_value=0)
        interpret_values.append(temp.reshape([new_height, new_width])) 
    return np.stack(interpret_values, axis = 2) 

In [12]:
import math

In [13]:
def normalize_image(im):
    return (im - np.amin(im)) / (np.amax(im) - np.amin(im))

Image Rectification

In [20]:
rectify_im1 = io.imread('images/rectify_im1.png')
rectify_im2 = io.imread('images/rectify_im2.png')

In [21]:
rectify_im3 = io.imread('images/rectify_im3.png')

In [22]:
f = open("rectify_im1_rectify_im1.json")
json_data = json.load(f) 
rectify_im1_points = np.array(json_data['im1Points'])
width = abs(np.min(rectify_im1_points, axis=0)[0] - np.max(rectify_im1_points, axis=0)[0])
height = abs(np.min(rectify_im1_points, axis=0)[1] - np.max(rectify_im1_points, axis=0)[1])
starting_point = np.min(rectify_im1_points, axis=0)[:2]
im1_rectified_points = np.asarray([starting_point, [starting_point[0] + width, starting_point[1]], [starting_point[0] + width, starting_point[1] + height], [starting_point[0], starting_point[1] + height]])

In [23]:
f = open("rectify_im3_rectify_im3.json")
json_data = json.load(f) 
rectify_im3_points = np.array(json_data['im1Points'])
width = abs(np.min(rectify_im3_points, axis=0)[0] - np.max(rectify_im3_points, axis=0)[0])
height = abs(np.min(rectify_im3_points, axis=0)[1] - np.max(rectify_im3_points, axis=0)[1])
starting_point = np.min(rectify_im3_points, axis=0)[:2]
im3_rectified_points = np.asarray([starting_point, [starting_point[0] + width, starting_point[1]], [starting_point[0] + width, starting_point[1] + height], [starting_point[0], starting_point[1] + height]])

Blend the images into a mosaic

In [28]:
f = open("ihouse1_ihouse2.json")
json_data = json.load(f)
ihouse1 = io.imread('images/ihouse1.png')
ihouse2 = io.imread('images/ihouse2.png')
ihouse1_points = np.array(json_data['im1Points'])
ihouse2_points = np.array(json_data['im2Points']) 

In [29]:
ihouseH = computeH(ihouse1_points, ihouse2_points)

/var/folders/lf/lddj8h6s3m3g8fkfp14cq4bw0000gn/T/ipykernel_71220/628019653.py:13: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  H_flatten = np.linalg.lstsq(A, B)[0]


In [30]:
ihousewarped = warp_image(ihouse1,ihouseH, "linear")

In [31]:
ihousewarped = normalize_image(ihousewarped[0])

/var/folders/lf/lddj8h6s3m3g8fkfp14cq4bw0000gn/T/ipykernel_71220/1012367921.py:2: RuntimeWarning: invalid value encountered in divide
  return (im - np.amin(im)) / (np.amax(im) - np.amin(im))


In [32]:
import math
import numpy as np
import matplotlib.pyplot as plt
import skimage.transform as sktr

def get_points(im1, im2):
    print('Please select 2 points in each image for alignment.')
    plt.imshow(im1)
    p1, p2 = plt.ginput(2)
    plt.close()
    plt.imshow(im2)
    p3, p4 = plt.ginput(2)
    plt.close()
    return (p1, p2, p3, p4)

def recenter(im, r, c):
    R, C, _ = im.shape
    rpad = (int) (np.abs(2*r+1 - R))
    cpad = (int) (np.abs(2*c+1 - C))
    return np.pad(
        im, [(0 if r > (R-1)/2 else rpad, 0 if r < (R-1)/2 else rpad),
             (0 if c > (C-1)/2 else cpad, 0 if c < (C-1)/2 else cpad),
             (0, 0)], 'constant')

def find_centers(p1, p2):
    cx = np.round(np.mean([p1[0], p2[0]]))
    cy = np.round(np.mean([p1[1], p2[1]]))
    return cx, cy

def align_image_centers(im1, im2, pts):
    p1, p2, p3, p4 = pts
    h1, w1, b1 = im1.shape
    h2, w2, b2 = im2.shape

    cx1, cy1 = find_centers(p1, p2)
    cx2, cy2 = find_centers(p3, p4)

    im1 = recenter(im1, cy1, cx1)
    im2 = recenter(im2, cy2, cx2)
    return im1, im2

def rescale_images(im1, im2, pts):
    p1, p2, p3, p4 = pts
    len1 = np.sqrt((p2[1] - p1[1])**2 + (p2[0] - p1[0])**2)
    len2 = np.sqrt((p4[1] - p3[1])**2 + (p4[0] - p3[0])**2)
    dscale = len2/len1
    if dscale < 1:
        im1 = sktr.rescale(im1, (dscale, dscale, 1))
    else:
        im2 = sktr.rescale(im2, (1./dscale, 1./dscale, 1))
    return im1, im2

def rotate_im1(im1, im2, pts):
    p1, p2, p3, p4 = pts
    theta1 = math.atan2(-(p2[1] - p1[1]), (p2[0] - p1[0]))
    theta2 = math.atan2(-(p4[1] - p3[1]), (p4[0] - p3[0]))
    dtheta = theta2 - theta1
    im1 = sktr.rotate(im1, dtheta*180/np.pi)
    return im1, dtheta

def match_img_size(im1, im2):
    # Make images the same size
    h1, w1, c1 = im1.shape
    h2, w2, c2 = im2.shape
    if h1 < h2:
        im2 = im2[int(np.floor((h2-h1)/2.)) : -int(np.ceil((h2-h1)/2.)), :, :]
    elif h1 > h2:
        im1 = im1[int(np.floor((h1-h2)/2.)) : -int(np.ceil((h1-h2)/2.)), :, :]
    if w1 < w2:
        im2 = im2[:, int(np.floor((w2-w1)/2.)) : -int(np.ceil((w2-w1)/2.)), :]
    elif w1 > w2:
        im1 = im1[:, int(np.floor((w1-w2)/2.)) : -int(np.ceil((w1-w2)/2.)), :]
    assert im1.shape == im2.shape
    return im1, im2

def align_images(im1, im2):
    pts = get_points(im1, im2)
    im1, im2 = align_image_centers(im1, im2, pts)
    im1, im2 = rescale_images(im1, im2, pts)
    im1, angle = rotate_im1(im1, im2, pts)
    im1, im2 = match_img_size(im1, im2)
    return im1, im2 

In [104]:
import PyQt5
%matplotlib qt5

In [147]:
aligned_ihouse1, aligned_ihouse2 = align_images(ihousewarped, ihouse2)

Please select 2 points in each image for alignment.


In [91]:
rectify_im1_warped = warp_image(rectify_im1, rectify_im1_H, 'linear')

done with color 0
done with color 1
done with color 2
done with color 3


In [34]:
from scipy.signal import convolve2d
def create_gaussian_stacks(im, N):
    image_pyramids = [] 
    temp = np.array([im[:,:,0], im[:,:,1], im[:,:,2]])
    image_pyramids.append(im)
    size, sigma = 6, 1
    while N > 0:
        GaussianFilter = np.outer(cv2.getGaussianKernel(size, sigma), cv2.getGaussianKernel(size, sigma))
        im_blurred_r = convolve2d(temp[0], GaussianFilter, "same")
        im_blurred_g = convolve2d(temp[1], GaussianFilter, "same")
        im_blurred_b = convolve2d(temp[2], GaussianFilter, "same")
        im_blurred = np.stack((im_blurred_r, im_blurred_g, im_blurred_b), axis=2)
        image_pyramids.append(im_blurred)
        temp = [im_blurred_r, im_blurred_g, im_blurred_b]
        size, sigma = size * 2, sigma * 2
        N -= 1 
    return np.array(image_pyramids)

def create_laplacian_stacks(im, N):
    gaussian_pyramids = create_gaussian_stacks(im, N)
    laplacian_pyramids = []
    for i in range(1, len(gaussian_pyramids)-1):
        laplacian_pyramids.append(gaussian_pyramids[i] - gaussian_pyramids[i-1])
    laplacian_pyramids.append(gaussian_pyramids[-1])
    return np.array(laplacian_pyramids) 

def create_gaussian_masks(mask, N, sigma):
    temp = [mask[:, :, 0], mask[:, :, 1], mask[:, :, 2]]
    mask_stacks = [mask]
    while N > 1:
        size = sigma * 6
        GaussianFilter = np.outer(cv2.getGaussianKernel(size, sigma), cv2.getGaussianKernel(size, sigma))
        mask_r = convolve2d(temp[0], GaussianFilter, "same")
        mask_g = convolve2d(temp[1], GaussianFilter, "same")
        mask_b = convolve2d(temp[2], GaussianFilter, "same")
        mask_stacked = np.stack((mask_r, mask_g, mask_b), axis=2)
        mask_stacks.append(mask_stacked)
        temp = [mask_r, mask_g, mask_b]
        sigma *= 2
        N -= 1 
    return np.array(mask_stacks) 

def combine_laplacian_stacks(im1, im2, N, binary_mask_right):
    laplacian_stack1 = create_laplacian_stacks(im1, N)
    laplacian_stack2 = create_laplacian_stacks(im2, N)
    mask_stack = create_gaussian_masks(binary_mask_right, N, 1)
    combined_stack = []
    for i in range(len(laplacian_stack1)):
        left = laplacian_stack1[i] * mask_stack[i]
        right = (np.ones(mask_stack[i].shape) - mask_stack[i]) * laplacian_stack2[i]
        combined_stack.append(left + right)
    return np.sum(combined_stack, axis = 0)



In [35]:
import cv2

In [272]:
ihouse1_aligned, ihouse2_aligned = align_images(ihouse1_warped, ihouse2_warped)

Please select 2 points in each image for alignment.


In [267]:
ihouse1_for_stitch = io.imread('images/ihouse1_aligned.png')
ihouse2_for_stitch = io.imread('images/ihouse2_aligned.png')
ihouse1_for_stitch, ihouse2_for_stitch = align_images(ihouse1_for_stitch, ihouse2_for_stitch)

Please select 2 points in each image for alignment.


In [328]:
def create_gaussian_stacks(im, N, sigma):
    image_pyramids = [] 
    temp = np.array([im[:,:,0], im[:,:,1], im[:,:,2]])
    image_pyramids.append(im) 
    size = sigma * 6
    while N > 0:
        GaussianFilter = np.outer(cv2.getGaussianKernel(size, sigma), cv2.getGaussianKernel(size, sigma))
        im_blurred_r = convolve2d(temp[0], GaussianFilter, "same", boundary="symm")
        im_blurred_g = convolve2d(temp[1], GaussianFilter, "same", boundary="symm")
        im_blurred_b = convolve2d(temp[2], GaussianFilter, "same", boundary="symm")
        im_blurred = np.stack((im_blurred_r, im_blurred_g, im_blurred_b), axis=2)
        image_pyramids.append(im_blurred)
        temp = [im_blurred_r, im_blurred_g, im_blurred_b]
        size, sigma = size * 2, sigma * 2 
        N -= 1 
    return np.array(image_pyramids)

def create_laplacian_stacks(im, N):
    gaussian_pyramids = create_gaussian_stacks(im, N, 1)
    laplacian_pyramids = []
    for i in range(len(gaussian_pyramids)-2):
        laplacian_pyramids.append(gaussian_pyramids[i] - gaussian_pyramids[i+1])
    laplacian_pyramids.append(gaussian_pyramids[-1])
    return np.array(laplacian_pyramids) 

In [329]:
def combine_laplacian_stacks(im1, im2, N, binary_mask_right):
    laplacian_stack1 = create_laplacian_stacks(im1, N)
    laplacian_stack2 = create_laplacian_stacks(im2, N)
    mask_stack = create_gaussian_masks(binary_mask_right, N, 5)
    combined_stack = []
    for i in range(len(laplacian_stack1)):
        left = laplacian_stack1[i] * mask_stack[i]
        right = (np.ones(mask_stack[i].shape) - mask_stack[i]) * laplacian_stack2[i]
        combined_stack.append(left + right)
    return np.sum(combined_stack, axis = 0)

In [374]:
f = open ("ihouse1_aligned_ihouse2_aligned.json")
json_data = json.load(f)
ihouse1_for_stitch_points = np.array(json_data['im1Points'])
ihouse2_for_stitch_points = np.array(json_data['im2Points'])
# mask = np.array(polygon(ihouse1_for_stitch_points[:, 0], ihouse1_for_stitch_points[:, 1])).T
# io.imshow(normalize_image(mask))
# plt.show()  
mask1 = polygon2mask(ihouse1_for_stitch.shape[:2], ihouse1_for_stitch_points[:, ::-1]).astype(np.uint8)
mask2 = polygon2mask(ihouse2_for_stitch.shape[:2], ihouse2_for_stitch_points[:, ::-1]).astype(np.uint8)
mask1_width = mask1.shape[1]
gray_value = 0.8
height = mask1.shape[0] 
width = mask1.shape[1]
print(width)
gray_mask_left = np.full((height, width*4//9), gray_value)
black_mask_right = np.zeros((height, width*5//9), dtype=np.uint8)
gray_mask_right = np.full((height, width*5//9), gray_value)
black_mask_left = np.zeros((height, width*4//9), dtype=np.uint8)
gray_right = np.concatenate((black_mask_left, gray_mask_right), axis=1)
gray_left = np.concatenate((gray_mask_left, black_mask_right), axis=1)
mask1_grayed = normalize_image(mask1)  
mask1_grayed = np.where(mask1_grayed == 1.0, 0.4444444444444445, mask1_grayed)
mask1_grayed = np.where(mask1_grayed == 0.0, 1.0, mask1_grayed)
mask1_grayed = np.where(mask1_grayed == 0.4444444444444445, 0.0, mask1_grayed)
mask2_grayed = normalize_image(mask2)   
mask2_grayed = np.where(mask2_grayed == 1.0, 0.4444444444444445, mask2_grayed)
mask2_grayed = np.where(mask2_grayed == 0.0, 1.0, mask2_grayed)
mask2_grayed = np.where(mask2_grayed == 0.4444444444444445, 0.0, mask2_grayed) 

738


In [375]:
ihouse1_for_stitch_resized = ihouse1_for_stitch[:, :, :3]
ihouse2_for_stitch_resized = ihouse2_for_stitch[:, :, :3]
mask1_grayed_converted = cv2.convertScaleAbs(mask1_grayed, alpha=255)
mask1_grayed_resized = cv2.cvtColor(mask1_grayed_converted, cv2.COLOR_GRAY2BGR)

In [376]:
mask2_grayed_converted = cv2.convertScaleAbs(mask2_grayed, alpha=255)
mask2_grayed_resized = cv2.cvtColor(mask2_grayed_converted, cv2.COLOR_GRAY2BGR)

In [391]:
mask1_grayed_finalized = mask1_grayed_resized /  255.0
mask1_grayed_finalized = 1 - mask1_grayed_finalized
mask1_grayed_finalized .shape

(396, 738, 3)

In [382]:
mask2_grayed_finalized = mask2_grayed_resized / 255.0
mask2_grayed_finalized = 1 - mask2_grayed_finalized

In [425]:
overlap = cv2.bitwise_and(mask1_grayed_finalized, mask2_grayed_finalized)
overlap_grayed = np.where(overlap == 1, 0.4, overlap)
overlap_blurred = cv2.GaussianBlur(overlap_grayed, (5, 5), 0)

In [440]:
mask_with_overlap = normalize_image(mask1_grayed_finalized - overlap_grayed)
mask_with_overlap_blurred = cv2.GaussianBlur(mask_with_overlap, (5, 5), 0)
mask_with_overlap_blurred.shape == mask1_grayed_finalized.shape

True

In [437]:
io.imshow(mask_with_overlap_blurred)
plt.show()

In [370]:
mask_combined = normalize_image(mask2_grayed_finalized + mask1_grayed_finalized)
# mask_combined = np.where(mask_combined != 0.7784313725490196, 0.0, mask_combined)
# # mask_combined = np.where(mask_combined == 0.6784313725490196, 0.9, mask_combined)
# # mask_combined = np.where(mask_combined != 0.9, 0.0, mask_combined)
io.imshow(normalize_image(mask_combined))
plt.show()

In [381]:
io.imshow(normalize_image(mask2_grayed_finalized))
plt.show()

In [379]:
mask_combined_blurred = cv2.GaussianBlur(mask_combined, (5, 5), 0)

In [380]:
mask_combined_finalized = normalize_image(mask_combined_blurred)

In [404]:
io.imshow(normalize_image(mask_combined_finalized))
plt.show()

In [400]:
mask_combined_finalized.shape == mask1_grayed_finalized.shape

True

In [405]:
io.imshow(normalize_image(mask1_grayed_finalized))
plt.show()

In [441]:
ihouse_combined = combine_laplacian_stacks(ihouse1_for_stitch_resized, ihouse2_for_stitch_resized, 5, mask_with_overlap_blurred)  

In [442]:
io.imshow(normalize_image(ihouse_combined))
plt.show() 

In [ ]:
f = open ("images/room1_room2.json")
json_data = json.load(f)
ihouse1_for_stitch_points = np.array(json_data['im1Points'])
ihouse2_for_stitch_points = np.array(json_data['im2Points'])
# mask = np.array(polygon(ihouse1_for_stitch_points[:, 0], ihouse1_for_stitch_points[:, 1])).T
# io.imshow(normalize_image(mask))
# plt.show()  
mask1 = polygon2mask(ihouse1_for_stitch.shape[:2], ihouse1_for_stitch_points[:, ::-1]).astype(np.uint8)
mask2 = polygon2mask(ihouse2_for_stitch.shape[:2], ihouse2_for_stitch_points[:, ::-1]).astype(np.uint8)
mask1_width = mask1.shape[1]
gray_value = 0.8
height = mask1.shape[0] 
width = mask1.shape[1]
print(width)
gray_mask_left = np.full((height, width*4//9), gray_value)
black_mask_right = np.zeros((height, width*5//9), dtype=np.uint8)
gray_mask_right = np.full((height, width*5//9), gray_value)
black_mask_left = np.zeros((height, width*4//9), dtype=np.uint8)
gray_right = np.concatenate((black_mask_left, gray_mask_right), axis=1)
gray_left = np.concatenate((gray_mask_left, black_mask_right), axis=1)
mask1_grayed = normalize_image(mask1)  
mask1_grayed = np.where(mask1_grayed == 1.0, 0.4444444444444445, mask1_grayed)
mask1_grayed = np.where(mask1_grayed == 0.0, 1.0, mask1_grayed)
mask1_grayed = np.where(mask1_grayed == 0.4444444444444445, 0.0, mask1_grayed)
mask2_grayed = normalize_image(mask2)   
mask2_grayed = np.where(mask2_grayed == 1.0, 0.4444444444444445, mask2_grayed)
mask2_grayed = np.where(mask2_grayed == 0.0, 1.0, mask2_grayed)
mask2_grayed = np.where(mask2_grayed == 0.4444444444444445, 0.0, mask2_grayed) 

In [551]:
room1 = io.imread('images/room1.png')
room2 = io.imread('images/room2.png')

In [552]:
f = open ("images/room1_room2.json")
json_data = json.load(f)
room1_for_stitch_points = np.array(json_data['im1Points'])
room2_for_stitch_points = np.array(json_data['im2Points'])

In [553]:
roomH = computeH(room1_for_stitch_points, room2_for_stitch_points)

/var/folders/lf/lddj8h6s3m3g8fkfp14cq4bw0000gn/T/ipykernel_71220/628019653.py:13: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  H_flatten = np.linalg.lstsq(A, B)[0]


In [554]:
room1_warped = warp_image(room1, roomH, 'linear')

done with color 0
done with color 1
done with color 2
done with color 3


In [555]:
io.imshow(normalize_image(room1_warped[0]))

In [532]:
room2_warped = warp_image(room2, roomH, 'linear')

done with color 0
done with color 1
done with color 2
done with color 3


In [457]:
room1_warped[0].shape

(500, 699, 4)

In [533]:
room1_warped_img = (room1_warped[0] * 255).astype(np.uint8)
room2_warped_img = (room2_warped[0] * 255).astype(np.uint8) 

In [539]:
io.imshow(normalize_image(room2_warped[0]))

In [556]:
room1_warped_img = normalize_image(room1_warped[0])

In [606]:
room1_aligned, room2_aligned = io.imread('room1_aligned.png'), io.imread('room2_aligned.png')

In [560]:
io.imshow(room1_aligned)

In [607]:
f = open ("room1_aligned_room2_aligned (1).json")
json_data = json.load(f)
room1_for_stitch_points = np.array(json_data['im1Points'])
room2_for_stitch_points = np.array(json_data['im2Points'])
mask1 = polygon2mask(room1_aligned.shape[:2], room1_for_stitch_points[:, ::-1]).astype(np.uint8)
mask2 = polygon2mask(room2_aligned.shape[:2], room2_for_stitch_points[:, ::-1]).astype(np.uint8)
mask1 = normalize_image(mask1)  
mask2 = normalize_image(mask2)   

In [598]:
room1_for_stitch_points 

array([[  2,   0],
       [167,   2],
       [350,  40],
       [340, 271],
       [  1, 272]])

In [486]:
# io.imshow(mask1)
# plt.show()

In [608]:
room1_for_stitch_resized = room1_aligned[:, :, :3]
room2_for_stitch_resized = room2_aligned[:, :, :3]
mask1_converted = cv2.convertScaleAbs(mask1, alpha=255)
mask1_resized = cv2.cvtColor(mask1_converted, cv2.COLOR_GRAY2BGR)
mask1_grayed_finalized = normalize_image(mask1_resized / 255.0) 
mask2_converted = cv2.convertScaleAbs(mask2, alpha=255)
mask2_resized = cv2.cvtColor(mask2_converted, cv2.COLOR_GRAY2BGR)
mask2_grayed_finalized = normalize_image(mask2_resized / 255.0) 

In [609]:
io.imshow(normalize_image(room2_for_stitch_resized))

In [610]:
io.imshow(normalize_image(mask2_grayed_finalized))

In [611]:
room1_combined = combine_laplacian_stacks(room1_for_stitch_resized, room2_for_stitch_resized, 5, mask1_grayed_finalized)

In [612]:
io.imshow(normalize_image(room1_combined))
plt.show() 